In [1]:
import numpy as np
import pandas as pd

import sys

from os.path import expanduser
sys.path.append(expanduser('~') + '/Lab/Utils/Python/')

from Conversions.translate import *
from Strings.is_a import *

# Initial setup

In [2]:
paper_pmid = 23733464
paper_name = 'islahudin_avery_2013' 

In [3]:
datasets = pd.read_csv('extras/YeastPhenome_' + str(paper_pmid) + '_datasets_list.txt', sep='\t', header=None, names=['pmid', 'name'])

In [4]:
datasets.set_index('pmid', inplace=True)

# Load & process the data

In [5]:
original_data = pd.read_excel('raw_data/zac999102061sd2.xlsx', sheet_name='Initial CQ Screen', skiprows=3)

In [6]:
print('Original data dimensions: %d x %d' % (original_data.shape))

Original data dimensions: 5664 x 13


In [7]:
original_data.head()

,ORF,Plate,Row,Col,Control A,Control B,Chloroquine A,Chloroquine B,MEAN Control,MEAN +Chloroquine,Growth Ratio (GR),Plate Median,Adjusted GR
0,EMPTY,1,A,1,0.117,0.116,0.110,0.105,0.1165,0.1075,SLOW,1.277311,SLOW
1,YHL047C,1,A,2,0.471,0.443,0.432,0.389,0.4570,0.4105,1.11328,1.277311,0.871578
2,YHL046C,1,A,3,0.736,0.509,0.392,0.424,0.6225,0.4080,1.52574,1.277311,1.19449
3,YHL045W,1,A,4,0.667,0.461,0.520,0.380,0.5640,0.4500,1.25333,1.277311,0.981228
4,YHL044W,1,A,5,0.117,0.119,0.107,0.107,0.1180,0.1070,SLOW,1.277311,SLOW


In [8]:
original_data['ORF'] = original_data['ORF'].astype(str)

In [9]:
# Eliminate all white spaces & capitalize
original_data['ORF'] = clean_orf(original_data['ORF'])

In [10]:
# Translate to ORFs 
original_data['ORF'] = translate_sc(original_data['ORF'], to='orf')

In [11]:
to_drop = original_data.loc[original_data['ORF']=='EMPTY',]

In [12]:
original_data.drop(index=to_drop.index, inplace=True)

In [13]:
# Make sure everything translated ok
t = looks_like_orf(original_data['ORF'])
print(original_data.loc[~t,])

                  ORF  Plate Row Col  Control A  Control B  Chloroquine A  \
index_input                                                                 
5604         WTBY4743     71   D   1      0.082      0.081          0.085   
5605         WTBY4743     71   D   2      0.083      0.081          0.083   

             Chloroquine  B  MEAN Control  MEAN +Chloroquine  \
index_input                                                    
5604                  0.084        0.0815             0.0845   
5605                  0.083        0.0820             0.0830   

            Growth Ratio (GR)  Plate Median Adjusted GR  
index_input                                              
5604                     SLOW           NaN        SLOW  
5605                     SLOW           NaN        SLOW  


In [14]:
original_data.drop(index=original_data.loc[~t,:].index, inplace=True)

In [15]:
# Eliminate the slow growin strains for which no accurate growth ratio could be calculated
to_drop = original_data.loc[original_data['Adjusted GR']=='SLOW',]

In [16]:
original_data.drop(index=to_drop.index, inplace=True)

In [17]:
original_data.shape

(4394, 13)

In [18]:
data = original_data[['ORF','Adjusted GR']].copy()

In [19]:
data.set_index('ORF', inplace=True)

In [20]:
data['Adjusted GR'] = data['Adjusted GR'].astype(float)

# Prepare the final dataset

In [21]:
dataset_ids = [16532]

In [22]:
datasets = datasets.reindex(index=dataset_ids)

In [23]:
data.columns = datasets['name'].values

In [24]:
data = data.groupby(data.index).mean()

In [25]:
# Create row index
data.index.name='orf'

In [26]:
print('Final data dimensions: %d x %d' % (data.shape))

Final data dimensions: 4251 x 1


# Print out

In [27]:
data.to_csv(paper_name + '.txt', sep='\t')

# Save to DB

In [28]:
from IO.save_data_to_db2 import *

In [29]:
# Create column index
lst = [datasets.index.values, datasets['name'].values]
tuples = list(zip(*lst))
idx = pd.MultiIndex.from_tuples(tuples, names=['dataset_id','dataset_name'])
data.columns = idx

In [30]:
save_data_to_db(data, paper_pmid)